#### List of IoT Sensors:

- GPS Sensor: To track the real-time location of the vehicle.
- Passenger Counting Sensor: To count the number of passengers entering and exiting the vehicle.
- Internet Connectivity Module: Cellular or Wi-Fi module to enable communication with the transit information platform.

#### Python Script:

- The python script is used to simulate the IoT sensors and generate the data.

##### To connect to the MQTT Broker:
- Start the MQTT Broker on the local machine.
- Run the python script to connect to the MQTT Broker.
- The python script will publish the data to the MQTT Broker.
- Subscribe to the MQTT Broker to receive the data.
- Command : `mosquitto_sub -t pto_data -d -h localhost`

##### Packages:

In [ ]:
%pip install paho-mqtt gps

##### Python Script:

In [ ]:
import time
import json
import paho.mqtt.client as mqtt
# import gps 

# Function to get real-time GPS data
def get_location_data():
    try:
        # session = gps.gps("localhost", "2947")
        # session.stream(gps.WATCH_ENABLE | gps.WATCH_NEWSTYLE)
        # location_data = None
        # while True:
        #     report = session.next()
        #     if report['class'] == 'TPV':
        #         latitude = getattr(report, 'lat', 0.0)
        #         longitude = getattr(report, 'lon', 0.0)
        #         location_data = {"latitude": latitude, "longitude": longitude}
        #         break
        location_data = {"latitude": 1.3521, "longitude": 103.8198} # Sample Data
        return location_data
    except Exception as e:
        print(f"Error getting location data: {str(e)}")
        return None

# Function to simulate ridership data
def get_ridership_data():
    try:
        passengers_in = 5
        passengers_out = 2
        return {"in": passengers_in, "out": passengers_out}
    except Exception as e:
        print(f"Error getting ridership data: {str(e)}")
        return None

# Initialize MQTT client
mqtt_client = mqtt.Client(protocol=mqtt.MQTTv5)
mqtt_client.connect("localhost", 1883)

while True:
    try:
        location_data = get_location_data()
        ridership_data = get_ridership_data()

        if location_data is not None and ridership_data is not None:
            payload = {
                "location": location_data,
                "ridership": ridership_data
            }

            # Send data to the transit information platform
            topic = "pto_data"
            mqtt_client.publish(topic, json.dumps(payload))
            
            print("Data sent:", payload)

        time.sleep(10)

    except Exception as e:
        print(f"Error: {str(e)}")
    finally: 
        mqtt_client.disconnect()

